In [81]:
import pandas as pd

# Initial inspection shows that the timestamp column is Unix epoch time up to the Second granularity
dateparse = lambda timestamp: pd.to_datetime(timestamp, unit='s')

data = pd.read_csv("seq/sample.csv", 
                    index_col="timestamp",
                    date_parser=dateparse,
                    infer_datetime_format=True,
                    parse_dates=['timestamp'])

In [82]:
# View a sample of 10 rows
data.sample(10)

,event_id,class,f1,f2,f3,f3.1,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-04-24 09:17:36,819703,0,1,1,1,0.000000,5,2,903.000000,3,518.791481,1,45,0,34,9604,1,81.000000,0,59.0,1.0,0,1.0,1,0,0,0,0,0,0,0,0,0
2018-09-03 15:07:13,824942,0,1,1,1,0.000000,2,2,1172.000000,7,651.034271,1,30,0,23,9822,0,0.000000,0,0.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0
2017-12-15 10:19:55,814722,0,1,2,3,1.039721,5,3,3356.000000,27,388.834757,3,124,0,63,9349,1,26.000000,0,5.0,0.0,0,0.0,1,0,0,0,0,0,0,0,0,0
2018-09-11 15:10:08,825500,1,1,45,111,2.266101,1767,1908,556.396396,86,306.422382,111,5078,0,4772,9287,1,646.000000,0,638.0,0.0,0,0.0,1,0,0,0,0,0,0,0,0,0
2017-11-17 08:39:02,813877,0,1,1,2,0.500402,1,9,1843.000000,3,360.764699,2,898,0,892,9246,0,0.000000,0,0.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0
2018-08-14 14:35:02,824050,1,2,3,3,0.780578,25,36,42.333333,11,177.627164,3,1168,0,1164,9438,2,800.500000,0,799.0,0.0,0,0.0,2,0,0,0,0,0,0,0,0,0
2017-11-14 14:06:16,813755,0,1,3,3,1.098612,3,3,424.000000,4,357.991944,3,340,0,245,9259,1,329.000000,0,324.0,0.0,0,0.0,1,0,0,0,0,0,0,0,0,0
2017-12-19 11:12:21,814801,1,1,37,120,3.594816,10428,0,0.225000,3,0.642793,1,2,0,0,9274,1,7899.000000,0,7555.0,7510.0,0,2.0,1,0,0,0,0,0,0,0,0,0
2018-07-19 17:01:24,822905,1,1,8,22,2.607902,111,289,477.227273,9,28.201409,22,815,0,591,9396,1,192.000000,0,183.0,0.0,0,0.0,1,0,0,0,0,0,0,0,0,0


In [83]:
# Inspect distribution of data
data.describe()

,event_id,class,f1,f2,f3,f3.1,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30
count,18417.000000,18417.000000,18417.000000,18417.000000,18417.000000,18417.000000,1.841700e+04,1.841700e+04,18417.000000,18417.000000,18417.000000,18417.000000,18417.000000,18417.0,18417.000000,18417.000000,18417.000000,18417.000000,18417.0,18417.000000,18417.000000,18417.0,18417.000000,18417.000000,18417.0,18417.0,18417.0,18417.0,18417.0,18417.0,18417.0,18417.0,18417.0
mean,820275.000000,0.451539,1.083618,7.641527,46.805343,0.948172,5.282271e+03,1.876271e+03,662.242501,11.078732,230.087888,23.974589,6143.728675,0.0,3341.689743,8940.345387,0.953901,1417.993660,0.0,1023.482719,159.500034,0.0,51.891048,0.953901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,5316.674289,0.497660,0.408374,134.453030,1375.277466,1.077906,3.433018e+05,4.628975e+04,1103.655690,13.299995,192.331715,295.660614,21027.359733,0.0,11756.653978,1888.918706,0.765509,3499.505021,0.0,2666.322131,1175.169459,0.0,489.727322,0.765509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,811067.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,-142.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,16.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,815671.000000,0.000000,1.000000,1.000000,1.000000,0.000000,4.000000e+00,1.000000e+00,88.000000,3.000000,68.332280,1.000000,218.000000,0.0,141.000000,9272.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,820275.000000,0.000000,1.000000,2.000000,3.000000,0.691761,2.200000e+01,8.000000e+00,286.680000,7.000000,180.443727,2.000000,741.000000,0.0,521.000000,9343.000000,1.000000,232.000000,0.0,174.000000,0.000000,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,824879.000000,1.000000,1.000000,3.000000,8.000000,1.507725,1.200000e+02,4.800000e+01,771.500000,14.000000,358.026991,7.000000,2715.000000,0.0,1573.000000,9426.000000,1.000000,1011.000000,0.0,777.000000,0.000000,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,829483.000000,1.000000,9.000000,16257.000000,172188.000000,10.287316,4.534482e+07,3.758535e+06,26059.000000,163.000000,714.998044,16095.000000,207897.000000,0.0,205927.000000,9982.000000,12.000000,40045.000000,0.0,37189.000000,11669.000000,0.0,11516.000000,12.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
# Looks like the below features only have constant 0 for all rows f11, f16, f19, f22 - f30
const_col = ["f11", "f16", "f19"] + ["f{}".format(num) for num in range(22,31)]

# dropping these cols
data.drop(columns=const_col, inplace=True, errors='ignore')

In [87]:
# Read resource dataset

res = pd.read_csv("seq/res.csv")
res.sample(10)

,event_id,res_id
577827,817427,15328352
837332,826451,15543884
660279,819373,15676460
517905,815574,15622573
673396,820269,15683279
360906,813061,15500144
491371,815050,15611669
685823,820499,15558365
457159,814773,15548465
422211,814258,15353043


In [88]:
import numpy as np

def group_res_by_event(df):
    keys, values = df.sort_values('event_id').values.T
    ukeys, index = np.unique(keys, True)
    arrays = np.split(values, index[1:])
    df2 = pd.DataFrame({'event_id':ukeys, 'res_id':[list(a) for a in arrays]})
    return df2

res_grp = group_res_by_event(res)
res_grp

,event_id,res_id
0,811067,"[15325277, 15440054, 15440055, 15440056, 15440..."
1,811068,[15495262]
2,811069,"[15501813, 15501809, 15501812, 15501810, 15501..."
3,811070,"[15505228, 15505227, 15505225, 15505229, 15505..."
4,811071,"[15449230, 15449260, 15449270, 15449271, 15449..."
...,...,...
18303,829479,"[15475824, 15475827, 15475828, 15475830, 15475..."
18304,829480,"[15658697, 15658696]"
18305,829481,"[15666344, 15632590, 15548015, 15666343, 15632..."
18306,829482,"[15658538, 15695471, 15658424]"


In [89]:
# Merging 2 dataset
data_w_res = data.reset_index().merge(res_grp, on="event_id", how="left").set_index("timestamp")

# Fill event with no resource an empty resource list
data_w_res['res_id'] = data_w_res['res_id'].apply(lambda d: d if isinstance(d, list) else [])

data_w_res.head(10)

,event_id,class,f1,f2,f3,f3.1,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f17,f18,f20,f21,res_id
timestamp,,,,,,,,,,,,,,,,,,,,,,
2016-11-21 14:17:52,811067,1,9,16257,172188,10.287316,45344815,0,0.000000,1,0.000000,0,0,0,9230,0,0.0,0.0,0.0,0.0,0,"[15325277, 15440054, 15440055, 15440056, 15440..."
2016-12-01 16:23:01,811068,0,1,1,1,0.000000,2,1,3.000000,2,10.086910,1,0,0,9231,0,0.0,0.0,0.0,0.0,0,[15495262]
2016-12-02 09:14:02,811069,1,7,1731,10212,7.460379,2000330,84524,157.627693,1,2.579979,2442,172188,172076,9230,0,0.0,0.0,0.0,0.0,0,"[15501813, 15501809, 15501812, 15501810, 15501..."
2016-12-02 10:18:05,811070,0,1,4,12,1.665603,353,0,0.000000,1,0.000000,0,0,0,9232,0,0.0,0.0,0.0,0.0,0,"[15505228, 15505227, 15505225, 15505229, 15505..."
2016-12-02 10:18:39,811071,0,2,4,22,0.919741,6263,4346,684.409091,2,10.341428,21,12,0,9232,0,0.0,0.0,0.0,0.0,0,"[15449230, 15449260, 15449270, 15449271, 15449..."
2016-12-02 15:35:26,811072,0,1,1,3,0.793341,31,33,1277.000000,2,11.053866,3,34,17,9232,0,0.0,0.0,0.0,0.0,0,"[15449271, 15449314, 15449270]"
2016-12-02 16:58:26,811073,0,1,1,1,0.000000,20,10,1103.000000,2,11.111505,1,37,20,9232,0,0.0,0.0,0.0,0.0,0,[15449318]
2016-12-08 17:07:09,811074,0,1,1,2,0.070304,119,108,952.500000,2,17.117558,2,38,21,9232,0,0.0,0.0,0.0,0.0,0,"[15449318, 15449317]"
2016-12-09 13:43:08,811075,0,1,3,9,1.330891,260,185,670.111111,2,17.975880,9,40,23,9232,0,0.0,0.0,0.0,0.0,0,"[15449270, 15449271, 15449272, 15449273, 15449..."


In [90]:
y = data_w_res[["class"]]

# Verify that the index column is of datetime type
data_w_res.index

DatetimeIndex(['2016-11-21 14:17:52', '2016-12-01 16:23:01',
               '2016-12-02 09:14:02', '2016-12-02 10:18:05',
               '2016-12-02 10:18:39', '2016-12-02 15:35:26',
               '2016-12-02 16:58:26', '2016-12-08 17:07:09',
               '2016-12-09 13:43:08', '2016-12-12 13:30:33',
               ...
               '2018-11-06 13:53:39', '2018-11-06 14:06:58',
               '2018-11-06 14:09:39', '2018-11-06 14:12:27',
               '2018-11-06 14:12:42', '2018-11-06 14:15:03',
               '2018-11-06 14:16:49', '2018-11-06 14:17:33',
               '2018-11-06 14:35:56', '2018-11-06 14:36:37'],
              dtype='datetime64[ns]', name='timestamp', length=18417, freq=None)

In [95]:
# Example of all events in the year 2018
data_w_res.loc['2018']

,event_id,class,f1,f2,f3,f3.1,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f17,f18,f20,f21,res_id
timestamp,,,,,,,,,,,,,,,,,,,,,,
2018-01-02 10:14:44,814967,0,1,1,1,0.000000,19,0,627.000000,6,406.831157,1,5,4,9352,0,0.0,0.0,0.0,0.0,0,[15486546]
2018-01-02 10:24:31,814968,0,1,1,1,0.000000,7,9,577.000000,2,105.876690,1,502,499,9343,1,277.0,277.0,0.0,0.0,1,[15562362]
2018-01-02 10:52:10,814969,0,1,1,1,0.000000,1,1,97.000000,2,406.857153,1,1008,52,9244,0,0.0,0.0,0.0,0.0,0,[15474689]
2018-01-02 11:01:08,814970,0,1,1,1,0.000000,3,41,212.000000,1,10.775336,1,277,277,9382,1,585.0,581.0,0.0,0.0,1,[15586003]
2018-01-02 11:01:44,814971,1,2,3,8,0.926948,130,18,4849.875000,11,254.289873,5,18,18,9434,0,0.0,0.0,0.0,0.0,0,"[15586006, 15586008, 15586007, 15435405, 15435..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-06 14:15:03,829479,0,1,1,6,1.149620,219,100,604.666667,2,714.998044,6,2430,1298,9489,1,364.0,248.0,0.0,0.0,1,"[15475824, 15475827, 15475828, 15475830, 15475..."
2018-11-06 14:16:49,829480,0,1,1,2,0.200000,196,2,49.500000,5,237.149051,2,41,41,435,1,620.0,619.0,0.0,0.0,1,"[15658697, 15658696]"
2018-11-06 14:17:33,829481,0,1,3,5,1.105769,58,36,973.800000,25,272.355479,5,2380,2373,9313,1,5502.0,5459.0,0.0,0.0,1,"[15666344, 15632590, 15548015, 15666343, 15632..."


In [91]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint